In [1]:
# importar paquetes importantes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas_profiling
from pandas_profiling import ProfileReport
from matplotlib import rcParams
import warnings

warnings.filterwarnings("ignore")

In [2]:
#Cargo los archivos .parquet que se nos fueron proporcionados, tanto el de train como el de test
df_train = pd.read_parquet("train.parquet")
df_test = pd.read_parquet("test.parquet")


## EDA y TRANSFORMACIONES

In [ ]:
df_train.shape 

In [ ]:
df_train.head()

In [ ]:
df_test.shape


In [ ]:
df_test.head()

## Realizacion del "ProfileReport

In [3]:
train_profile = ProfileReport(df_train, title="Train", minimal=True)
test_profile = ProfileReport(df_test, title="Test")

In [4]:
test_profile.to_file("test_profile.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
train_profile.to_file("train_profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Transformacion de los dataframes, creacion de la columna PRED para guardar las predicciones y limpieza de las columnas que decidi que son innecesarias

In [6]:
#drop id, url, region url, laundry, parking, description, image url, lat, long
df_train['pred'] = df_train['price'].apply(lambda x: 1 if x < 999 else 0)
df_test['pred'] = None
df_train.drop(['id','url','region_url', 'laundry_options', 'parking_options', 'image_url','lat', 'long', 'description' ], axis= 1, inplace= True)
df_test.drop(['id','url','region_url', 'laundry_options', 'parking_options', 'image_url','lat', 'long', 'description' ], axis= 1, inplace= True)

Transformacion de las columnas categóricas en valores numéricos

In [7]:
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

en el siguiente paso definimos cuáles son las columnas que se encuentran tanto en el set de entrenamiento como en el set de testeo, para poder tener las mismas columnas en los dos sets y que el entrenamiento se realice correctamente. 

In [8]:
columns_to_use = set(df_train.columns).intersection(set(df_test.columns))


Entrenamiento del modelo. separamos, por un lado la COLUMNA PRED como Y y el resto de columnas como X

In [9]:
X = df_train[columns_to_use]
y = df_train.pred

In [10]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Para medir la precision del entrenamiento voy a realizar un TRAIN TEST SPLIT, separando en dos partes el set de entrenamiento asi pudiendo comparar qué tan preciso fue el dicho entrenamiento y pudiendo asi ajustar mejor los parámetros.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)

In [12]:
clf = RandomForestClassifier(n_estimators=1550, n_jobs=-1, random_state = 42, max_features='sqrt')
clf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=550, n_jobs=-1, random_state=42)

In [13]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_train, y_train_pred))
print(metrics.accuracy_score(y_test, y_test_pred))

0.9810933057637633
0.9330674572980348


buenisimo, ya vimos que funciona bastaaaaante bien. ahora: vamos a predecir datos desconocidos 

In [14]:

X_new = df_test[list(columns_to_use)]

y_pred = clf.predict(X_new)


guardamos... y entregamos. 

In [15]:
df_y_pred = pd.DataFrame(y_pred,columns=["pred"])
df_y_pred.to_csv("Defo0.csv", index=False)

para buscar los mejores hiperparámetros utilicé la tecnica Grid Search, la cual prueba parametro por parametro y al final nos dice cuál da mejor resultado. el problema es que tarda un monton, y a veces da error despues de un tiempo (horas)

In [ ]:

from sklearn.model_selection import GridSearchCV

# Crea un objeto del modelo
rf_model = RandomForestClassifier()

# Define los parámetros a probar
param_grid = {
    #'n_estimators': [40,80,140,250],
    #'max_depth': [None, 5, 10, 20],
    #'min_samples_split': [2, 5, 10],
    #'min_samples_leaf': [1, 2, 4],
    #'max_features': ['sqrt', 'log2', None]
}

# Crea un objeto de GridSearchCV
grid_search = GridSearchCV(rf_model, param_grid, cv=2, n_jobs=-1, verbose=2, return_train_score=True)

# Entrena el modelo con los datos X e y
grid_search.fit(X_train, y_train)

# Imprime los mejores parámetros encontrados
print(grid_search.best_params_)

# Realiza las predicciones con el conjunto de datos de prueba
y_pred = grid_search.predict(X_new)